# Sequence to Sequence (a.k.a. seq2seq)

**학습목표**
* Encoder Decoder 구조를 이해하고 구현할 줄 안다.
* Seq2Seq에 필요한 전처리를 이해한다.

![이런거](https://raw.githubusercontent.com/KerasKorea/KEKOxTutorial/master/media/28_1.png)
---------------------------------
edu.rayleigh@gmail.com
Special Thanks to : 숙번님 ( [봉수골 개발자 이선비](https://www.youtube.com/channel/UCOAyyrvi7tnCAz7RhH98QCQ) )

In [ ]:
# !wget http://www.manythings.org/anki/fra-eng.zip

In [ ]:
# import zipfile
# fra_eng = zipfile.ZipFile('fra-eng.zip')
# fra_eng.extractall()
# fra_eng.close()

In [ ]:
!wget https://raw.githubusercontent.com/L1aoXingyu/seq2seq-translation/master/data/eng-fra.txt

In [ ]:
import pandas as pd
# temp = pd.read_table('fra.txt', names=['Eng', 'Fra', 'License'])
temp = pd.read_table('eng-fra.txt', names=['Eng', 'Fra'])
temp.shape

In [ ]:
temp.head()

# 너무 많으므로 50000개 문장만 진행하자.

In [ ]:
# ## 끔찍한 결과를 볼 수 있다.
# temp = temp.sample(n=50000, replace=False, random_state=2021)

temp = temp.iloc[:50000]

In [ ]:
eng_sent = temp['Eng'].tolist()
fra_sent = temp['Fra'].tolist()

In [ ]:
print(eng_sent[100])
print(fra_sent[100])

# 데이터 준비
0. 단어와 구두점 사이 공백 만들기
1. sos 와 eos
1. tokenizing, idx_seq, padding

## 0. 단어와 구두점 사이 공백 만들기


In [ ]:
import unicodedata
import re
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')
def preprocess_sentence(sent):
    # 위에서 구현한 함수를 내부적으로 호출
    sent = unicode_to_ascii(sent.lower())

    # 단어와 구두점 사이에 공백을 만듭니다.
    # Ex) "he is a boy." => "he is a boy ."
    sent = re.sub(r"([?.!,'¿])", r" \1 ", sent)

    # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환합니다.
    sent = re.sub(r"[^a-zA-Z!.?']+", r" ", sent)

    sent = re.sub(r"\s+", " ", sent)
    return sent

In [ ]:
preprocess_sentence("I'm just a poor boy.")

In [ ]:
eng_sent = [ preprocess_sentence(sent) for sent in eng_sent ]
fra_sent = [ preprocess_sentence(sent) for sent in fra_sent ]

In [ ]:
print(eng_sent[100])
print(fra_sent[100])

## 1. sos 와 eos
1. sos : start of speech
2. eos : end of speech

In [ ]:
fra_sent = [f"<sos> {fra} <eos>" for fra in fra_sent]
fra_sent[100]

## 2. Tokenizing, idx_seq, padding

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Tokenizing
tokenizer_en = Tokenizer(filters="", lower=True)
tokenizer_en.fit_on_texts(eng_sent)
tokenizer_fr = Tokenizer(filters="", lower=True)
tokenizer_fr.fit_on_texts(fra_sent)

In [ ]:
# Index Sequence
eng_seq = tokenizer_en.texts_to_sequences(eng_sent)
fra_seq = tokenizer_fr.texts_to_sequences(fra_sent)

print(eng_seq[100])
print(fra_seq[100])

In [ ]:
# padding
eng_pad = pad_sequences(eng_seq) # 최대 문장 길이에 패딩에 맞춰지게 됨.
fra_pad = pad_sequences(fra_seq)

print(eng_pad.shape)
print(fra_pad.shape)

In [ ]:
# tokenizer에서 0 index가 구성되어있지 않지만, 
# pad_sequence에서 pad의 의미로 0을 사용하고 있어서, 전체 사이즈를 구할 때, +1을 해준다.

eng_vocab_size = len(tokenizer_en.word_index) + 1
fra_vocab_size = len(tokenizer_fr.word_index) + 1
print("영어 단어 집합의 크기: {:d}\n프랑스어 단어 집합의 크기: {:d}".format(eng_vocab_size, fra_vocab_size))

# 모델링!

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, GRU

In [ ]:
# 혹시 이미 그려둔 그래프가 있다면 날려줘!
tf.keras.backend.clear_session()

# 영어 단어 집합의 크기 : 5965, (50000, 11)
# 프랑스어 단어 집합의 크기 : 10406, (50000, 19)
# 프랑스어 문장은 길이가 19이지만,
# 디코더의 인풋으로 넣을때는 맨 뒤의 <eos>를 떼고 길이 18의 문장을
# 디코더의 아웃풋은 맨 앞의 <eos>를 떼고 길이 18의 문장으로 준비해야 함.

# Encoder
enc_X = tf.keras.layers.Input(shape=[eng_pad.shape[1]])
enc_E = tf.keras.layers.Embedding(eng_vocab_size, 64)(enc_X) # 토큰수, 차원수
enc_S_full, enc_S = tf.keras.layers.GRU(256, return_sequences=True, return_state=True)(enc_E)
## 물론 지금은 enc_S_full은 사용하지 않는다.

# Decoder
dec_X = tf.keras.layers.Input(shape=[fra_pad.shape[1]-1])
dec_E = tf.keras.layers.Embedding(fra_vocab_size, 64)(dec_X) # 토큰수, 차원수
dec_H = tf.keras.layers.GRU(256, return_sequences=True)(dec_E, initial_state=enc_S)
dec_H = tf.keras.layers.Dense(256, activation="swish")(dec_H) # 없어도 상관은 없는 부분.
dec_Y = tf.keras.layers.Dense(fra_vocab_size, activation="softmax")(dec_H) # 매시점에서, 어떤 단어가 타당할지 분류 문제로 푸는 것

model = tf.keras.models.Model([enc_X, dec_X], dec_Y)
# 텍스트는 index이고(원핫인코딩을 안했고)
# 아웃풋레이어는 분류문제 처럼 노드가 준비되어 있다면
# sparse categorical crossentropy
model.compile(loss='sparse_categorical_crossentropy',
              optimizer = 'rmsprop',
              metrics=['accuracy'])
model.summary()

In [ ]:
# decoder의 인풋은 마지막 <eos>를 뗀다.
# decoder의 아웃풋 학습시엔 처음의 <sos>를 뗀다.
model.fit([eng_pad, fra_pad[:, :-1]], fra_pad[:, 1:], shuffle=True, 
          batch_size=128, epochs=10)

In [ ]:
import numpy as np

# 영어 단어 집합의 크기 : 5965, (50000, 11)
# 프랑스어 단어 집합의 크기 : 10406, (50000, 19)

def translate(eng):
    # eng => index => pad
    eng_seq = tokenizer_en.texts_to_sequences([eng])
    eng_pad = tf.keras.preprocessing.sequence.pad_sequences(eng_seq, maxlen=11)

    fra = []
    for n in range(19-1):
        # fra => index => pad
        fra_seq = tokenizer_fr.texts_to_sequences([['<sos>'] + fra])
        fra_pad = tf.keras.preprocessing.sequence.pad_sequences(fra_seq, maxlen=19-1)

        fra_next = model.predict([eng_pad, fra_pad])

        # onehot -> index -> word
        fra = [tokenizer_fr.index_word[i] for i in np.argmax(fra_next[0], axis=1) if i != 0]
        # 번역된 word 선택
        fra = fra[:n+1]
        
        if fra[-1] == '<eos>':
            break

    return fra

In [ ]:
import random

# 랜덤 10개
indices = list(range(50000))
random.shuffle(indices)

for n in indices[:10]:
    print(f"영어: {eng_sent[n]}\n불어: {fra_sent[n]}")
    print(f"번역: {' '.join(translate(eng_sent[n])[:-1])}")
    print()